In [1]:
# Create Model Using ResNet-50
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def resnet_block(inputs, filters, strides=1):
    x = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    
    if strides > 1:
        shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(inputs)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = inputs
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)
    
    x = resnet_block(x, filters=128, strides=2)
    x = resnet_block(x, filters=128)
    
    x = resnet_block(x, filters=256, strides=2)
    x = resnet_block(x, filters=256)
    x = resnet_block(x, filters=256)
    
    x = resnet_block(x, filters=512, strides=2)
    x = resnet_block(x, filters=512)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model

# Path ke folder dataset
dataset_path = '../Datasets'

# Parameter untuk model dan pelatihan
input_shape = (64, 64, 3)
num_classes = 12
batch_size = 32
epochs = 10

# Memuat dataset
train_data_dir = os.path.join(dataset_path + '/Train')
val_data_dir = os.path.join(dataset_path + '/Validation')
test_data_dir = os.path.join(dataset_path + '/Test')

# Mempersiapkan augmentasi gambar untuk data pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Mempersiapkan generator data pelatihan
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

# Mempersiapkan augmentasi gambar untuk data validasi dan pengujian
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Mempersiapkan generator data validasi dan pengujian
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

# Membuat model ResNet
model = create_resnet(input_shape, num_classes)

# Menentukan fungsi optimasi, fungsi loss, dan metrik evaluasi
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Melatih model dengan data generator
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size)

# Evaluasi model pada data pengujian
test_loss, test_acc = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size)

print('Test accuracy:', test_acc)

Found 8624 images belonging to 12 classes.
Found 1857 images belonging to 12 classes.
Found 1843 images belonging to 12 classes.
Epoch 1/10
269/269 [==============================] - 1082s 4s/step - loss: 1.2207 - accuracy: 0.5978 - val_loss: 5.5099 - val_accuracy: 0.1369
Epoch 2/10
269/269 [==============================] - 1008s 4s/step - loss: 0.7554 - accuracy: 0.7453 - val_loss: 0.9222 - val_accuracy: 0.6816
Epoch 3/10
269/269 [==============================] - 1058s 4s/step - loss: 0.5982 - accuracy: 0.7943 - val_loss: 0.9036 - val_accuracy: 0.7107
Epoch 4/10
269/269 [==============================] - 935s 3s/step - loss: 0.5159 - accuracy: 0.8188 - val_loss: 0.9582 - val_accuracy: 0.7031
Epoch 5/10
269/269 [==============================] - 835s 3s/step - loss: 0.4354 - accuracy: 0.8508 - val_loss: 0.7298 - val_accuracy: 0.7689
Epoch 6/10
269/269 [==============================] - 857s 3s/step - loss: 0.3691 - accuracy: 0.8735 - val_loss: 0.7217 - val_accuracy: 0.7602
Epoch 7/10

In [2]:
# Save model in format HDF5
h5_model_path = '../Model/frucheck-model.h5'

model.save(h5_model_path)

In [4]:
# Class Name
class_names = list(train_generator.class_indices.keys())
print(class_names)

['FreshApples', 'FreshBanana', 'FreshCarrot', 'FreshOranges', 'FreshPotato', 'FreshTomato', 'RottenApples', 'RottenBanana', 'RottenCarrot', 'RottenOranges', 'RottenPotato', 'RottenTomato']


In [17]:
### BATCH IMAGE PREDICTION ###

from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load model
model_path = "../Model/frucheck-model.h5"
loaded_model = load_model(model_path)

# Define class names
class_names = ['ApelSegar', 'PisangSegar', 'WortelSegar', 'JerukSegar', 'KentangSegar', 'TomatSegar', 'ApelBusuk', 'PisangBusuk', 'WortelBusuk', 'JerukBusuk', 'KentangBusuk', 'TomatBusuk']

# Evaluate predictions for each class
dataset_path = '../Datasets'
test_data_dir = os.path.join(dataset_path + '/Test')
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Perform predictions for each batch of the test generator
predictions = loaded_model.predict_generator(test_generator, steps=len(test_generator))

# Loop through each class
for i, class_name in enumerate(class_names):
    class_indices = np.where(test_generator.classes == i)[0]
    class_images = np.array(test_generator.filenames)[class_indices]
    
    # Loop through each image in the class
    for image_path in class_images:
        image = Image.open(os.path.join(test_data_dir, image_path))
        
        # Perform prediction for the image
        image_array = np.array(image.resize((64, 64))) / 255.0
        input_image = np.expand_dims(image_array, axis=0)
        predicted_class = np.argmax(predictions[class_indices[class_images == image_path]])
        predicted_label = class_names[predicted_class]
        
        # VARIABLE HASIL KLASIFIKASI
        label_predicted = predicted_label.replace('Segar', '').replace('Busuk', '') # Klasifikasi nama buah/sayur
        freshness = 'Segar' if 'Segar' in class_name else 'Busuk' # Menentukan segar atau busuk
        
        # Calculate prediction accuracy
        predicted_prob = np.max(predictions[class_indices[class_images == image_path]])
        accuracy = predicted_prob * 100 # Akurasi prediksi
        
        # Print the predicted class, freshness, and accuracy
        print('Image:', image_path)
        print('Predicted Class:', label_predicted)
        print('Freshness:', freshness)
        print('Accuracy: {:.2f}%'.format(accuracy))
        print('---')

Found 1843 images belonging to 12 classes.


C:\Users\muham\AppData\Local\Temp\ipykernel_18340\3164379828.py:27: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = loaded_model.predict_generator(test_generator, steps=len(test_generator))


Image: FreshApples\Apple_Fresh-1.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 97.19%
---
Image: FreshApples\Apple_Fresh-103_augmented.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.90%
---
Image: FreshApples\Apple_Fresh-104.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 85.43%
---
Image: FreshApples\Apple_Fresh-105.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 54.50%
---
Image: FreshApples\Apple_Fresh-106.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.73%
---
Image: FreshApples\Apple_Fresh-109_augmented.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.78%
---
Image: FreshApples\Apple_Fresh-111.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 88.64%
---
Image: FreshApples\Apple_Fresh-113_augmented.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 98.52%
---
Image: FreshApples\Apple_Fresh-118.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.15%
---
Image: FreshApples\Apple_Fresh-123_augmented.jpg
Predicted Class: Apel
Freshness:

Image: FreshBanana\Banana_Fresh-139.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshBanana\Banana_Fresh-143_augmented.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshBanana\Banana_Fresh-147.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshBanana\Banana_Fresh-148.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshBanana\Banana_Fresh-15.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 99.95%
---
Image: FreshBanana\Banana_Fresh-152.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 99.84%
---
Image: FreshBanana\Banana_Fresh-152_augmented.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 99.84%
---
Image: FreshBanana\Banana_Fresh-155_augmented.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshBanana\Banana_Fresh-160.jpg
Predicted Class: Pisang
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshBanana\Banana_Fresh-163_augmented.j

Image: FreshCarrot\Carrot_Fresh-183.jpg
Predicted Class: Wortel
Freshness: Segar
Accuracy: 93.66%
---
Image: FreshCarrot\Carrot_Fresh-189.jpg
Predicted Class: Wortel
Freshness: Segar
Accuracy: 81.55%
---
Image: FreshCarrot\Carrot_Fresh-201.jpg
Predicted Class: Wortel
Freshness: Segar
Accuracy: 91.66%
---
Image: FreshCarrot\Carrot_Fresh-203_augmented.jpg
Predicted Class: Wortel
Freshness: Segar
Accuracy: 97.17%
---
Image: FreshCarrot\Carrot_Fresh-209.jpg
Predicted Class: Wortel
Freshness: Segar
Accuracy: 99.22%
---
Image: FreshCarrot\Carrot_Fresh-20_augmented.jpg
Predicted Class: Wortel
Freshness: Segar
Accuracy: 97.68%
---
Image: FreshCarrot\Carrot_Fresh-212.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 55.45%
---
Image: FreshCarrot\Carrot_Fresh-212_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 40.91%
---
Image: FreshCarrot\Carrot_Fresh-213_augmented.jpg
Predicted Class: Wortel
Freshness: Segar
Accuracy: 53.04%
---
Image: FreshCarrot\Carrot_Fresh-216.jpg
Predi

...

Image: FreshTomato\Tomato_Fresh-69_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 98.14%
---
Image: FreshTomato\Tomato_Fresh-71_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 96.81%
---
Image: FreshTomato\Tomato_Fresh-72_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 99.99%
---
Image: FreshTomato\Tomato_Fresh-7_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 99.88%
---
Image: FreshTomato\Tomato_Fresh-82_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 92.54%
---
Image: FreshTomato\Tomato_Fresh-83.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 99.98%
---
Image: FreshTomato\Tomato_Fresh-85_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 99.99%
---
Image: FreshTomato\Tomato_Fresh-96.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 99.93%
---
Image: FreshTomato\Tomato_Fresh-96_augmented.jpg
Predicted Class: Tomat
Freshness: Segar
Accuracy: 99.91%
---
Image: FreshTomato\Tomato_Fresh

Image: RottenBanana\Banana_Rotten-162_augmented.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 99.96%
---
Image: RottenBanana\Banana_Rotten-163.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 100.00%
---
Image: RottenBanana\Banana_Rotten-165.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 99.96%
---
Image: RottenBanana\Banana_Rotten-166.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 95.78%
---
Image: RottenBanana\Banana_Rotten-171_augmented.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 99.96%
---
Image: RottenBanana\Banana_Rotten-173_augmented.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 100.00%
---
Image: RottenBanana\Banana_Rotten-175.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 100.00%
---
Image: RottenBanana\Banana_Rotten-175_augmented.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 99.99%
---
Image: RottenBanana\Banana_Rotten-178.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 95.68%
---
Image: RottenBanana\B

Image: RottenCarrot\Carrot_Rotten-214_augmented.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 55.52%
---
Image: RottenCarrot\Carrot_Rotten-218.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 69.66%
---
Image: RottenCarrot\Carrot_Rotten-223.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 90.74%
---
Image: RottenCarrot\Carrot_Rotten-226_augmented.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 91.58%
---
Image: RottenCarrot\Carrot_Rotten-227.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 59.00%
---
Image: RottenCarrot\Carrot_Rotten-228.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 64.22%
---
Image: RottenCarrot\Carrot_Rotten-228_augmented.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 50.95%
---
Image: RottenCarrot\Carrot_Rotten-229_augmented.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 61.07%
---
Image: RottenCarrot\Carrot_Rotten-235.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 99.64%
---
Image: RottenCarrot\Carro

Image: RottenOranges\Orange_Rotten-368_augmented.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 99.98%
---
Image: RottenOranges\Orange_Rotten-371_augmented.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 90.40%
---
Image: RottenOranges\Orange_Rotten-374.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 98.03%
---
Image: RottenOranges\Orange_Rotten-375.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 92.87%
---
Image: RottenOranges\Orange_Rotten-379_augmented.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 99.99%
---
Image: RottenOranges\Orange_Rotten-380.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 100.00%
---
Image: RottenOranges\Orange_Rotten-382.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 99.99%
---
Image: RottenOranges\Orange_Rotten-386.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 100.00%
---
Image: RottenOranges\Orange_Rotten-386_augmented.jpg
Predicted Class: Jeruk
Freshness: Busuk
Accuracy: 99.99%
---
Image: RottenOranges\O

Image: RottenPotato\Potato_Rotten-49.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 99.21%
---
Image: RottenPotato\Potato_Rotten-50.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 84.27%
---
Image: RottenPotato\Potato_Rotten-51_augmented.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 95.52%
---
Image: RottenPotato\Potato_Rotten-52_augmented.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 71.44%
---
Image: RottenPotato\Potato_Rotten-53.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 98.81%
---
Image: RottenPotato\Potato_Rotten-54.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 99.22%
---
Image: RottenPotato\Potato_Rotten-57_augmented.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 98.24%
---
Image: RottenPotato\Potato_Rotten-66_augmented.jpg
Predicted Class: Pisang
Freshness: Busuk
Accuracy: 84.18%
---
Image: RottenPotato\Potato_Rotten-6_augmented.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 98.84%
---
Image: RottenPot

In [20]:
### SINGLE IMAGE PREDICTION ###

from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model_path = "../Model/frucheck-model.h5"
loaded_model = load_model(model_path)

# Define class names
class_names = ['ApelSegar', 'PisangSegar', 'WortelSegar', 'JerukSegar', 'KentangSegar', 'TomatSegar', 'ApelBusuk', 'PisangBusuk', 'WortelBusuk', 'JerukBusuk', 'KentangBusuk', 'TomatBusuk']

# Load and preprocess the image
image_path = '../Datasets/Test/RottenCarrot/Carrot_Rotten-64_augmented.jpg'  # SOURCE IMAGE
image = Image.open(image_path)
image = image.resize((64, 64))
image_array = np.array(image) / 255.0
input_image = np.expand_dims(image_array, axis=0)

# Perform prediction
predictions = loaded_model.predict(input_image)
predicted_class = np.argmax(predictions[0])
predicted_label = class_names[predicted_class]

# VARIABLE HASIL KLASIFIKASI
label_predicted = predicted_label.replace('Segar', '').replace('Busuk', '') # Klasifikasi nama buah/sayur
freshness = 'Segar' if 'Segar' in class_name else 'Busuk' # Menentukan segar atau busuk

# Calculate prediction accuracy
predicted_prob = np.max(predictions[0])
accuracy = predicted_prob * 100

# Print the predicted class, freshness, and accuracy
print('Image:', image_path)
print('Predicted Class:', label_predicted)
print('Freshness:', freshness)
print('Accuracy: {:.2f}%'.format(accuracy))

1/1 [==============================] - 1s 1s/step
Image: ../Datasets/Test/RottenCarrot/Carrot_Rotten-64_augmented.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 99.62%


In [22]:
# Save model in format SavedModel
saved_model_path = '../Model'

tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: ../Model\assets


INFO:tensorflow:Assets written to: ../Model\assets


In [23]:
### FOR PREDICTION USING TF SERVING ###
import requests
import numpy as np
from PIL import Image

# URL endpoint TensorFlow Serving
url = "http://localhost:8501/v1/models/model_name:predict"  # endpoint URL

# Load and preprocess the image
image_path = "../Datasets/Test/RottenCarrot/Carrot_Rotten-64_augmented.jpg"  # Image path
image = Image.open(image_path)
image = image.resize((64, 64))
image_array = np.array(image) / 255.0
input_image = np.expand_dims(image_array, axis=0)

# Convert input to JSON format
input_data = {
    "instances": input_image.tolist()
}

# Send prediction request
response = requests.post(url, json=input_data)

# Get prediction result
predictions = np.array(response.json()["predictions"][0])

# Get predicted class index
predicted_class = np.argmax(predictions)
predicted_label = class_names[predicted_class]

# CLASSIFICATION RESULTS VARIABLES
label_predicted = predicted_label.replace('Segar', '').replace('Busuk', '') # Klasifikasi nama buah/sayur
freshness = 'Segar' if 'Segar' in class_name else 'Busuk' # Menentukan segar atau busuk

# Calculate prediction accuracy
predicted_prob = np.max(predictions)
accuracy = predicted_prob * 100

# Print the predicted class, freshness, and accuracy
print('Image:', image_path)
print('Predicted Class:', label_predicted)
print('Freshness:', freshness)
print('Accuracy: {:.2f}%'.format(accuracy))
